In [1]:
import gensim
from gensim.models import Word2Vec
import re

In [2]:
from gensim.models import word2vec

In [3]:
cleaned_tags = []
with open('artist_tags.csv', 'r') as inpFile:
    lines = inpFile.readlines()
    for line in lines:
        tag = line.replace("\n", "").replace("\"", "")
        tags = tag.split(",")
        cleaned_tags.append(tags)
    

In [4]:
len(cleaned_tags)

100136

In [5]:
cleaned_tags[0]

['rocket',
 'uniform',
 'villain',
 'grunt',
 'team rocket',
 'team rocket grunt',
 'pokemon']

In [6]:
#word2vec_cbow = Word2Vec(sentences=cleaned_tags, size=100, window=5, min_count=1, workers=4, sg=0, iter=30)

In [7]:
word2vec_sg = Word2Vec(sentences=cleaned_tags, size=300, window=5, min_count=1, workers=4, sg=1, iter=30)

In [8]:
#word2vec_cbow.most_similar(positive=['django', 'rocket'])

[('sid', 0.6566838026046753),
 ('unchained', 0.6359002590179443),
 ('tarantino', 0.6325922012329102),
 ('duct', 0.6305240392684937),
 ('and bottle', 0.6228089332580566),
 ('pizza planet', 0.6116124987602234),
 ('bounty hunter', 0.6088114976882935),
 ('exploitation', 0.6079063415527344),
 ('rc', 0.6069498658180237),
 ('aviator', 0.5953713655471802)]

In [9]:
word2vec_sg.most_similar(positive=['django', 'rocket'])

[('mono racer', 0.746355414390564),
 ('astronauta', 0.7307371497154236),
 ('tarentino', 0.719941258430481),
 ('pulpboutique', 0.7169310450553894),
 ('django unchained the d is silent hillbilly quentin tarantino blood gore slavery jamie foxx christoph walt leonardo dicaprio calvin candie inglourious basterds',
  0.7168609499931335),
 ('rocketeer', 0.7153801918029785),
 ('astronaunt', 0.7116128206253052),
 ('juanotron', 0.710219144821167),
 ('ruby73 and rabab elghamry', 0.7090080976486206),
 ('tarenttino', 0.7058366537094116)]

In [10]:
#word2vec_cbow.most_similar(positive=['rocket','uniform','villain','grunt','team rocket','team rocket grunt','pokemon'])

[('gym leader', 0.6617014408111572),
 ('pikachu', 0.6214253902435303),
 ('baddie', 0.6213517189025879),
 ('meowth', 0.5972244739532471),
 ('pokemon ash', 0.5960426926612854),
 ('corps', 0.5864928960800171),
 ('high pikachu', 0.5756170749664307),
 ('gym badge', 0.570639967918396),
 ('misty', 0.5695374011993408),
 ('bulbasaur', 0.560421347618103)]

In [11]:
word2vec_sg.most_similar(positive=['rocket','uniform','villain','grunt','team rocket','team rocket grunt','pokemon'])

[('rocket grunt', 0.8055406808853149),
 ('rocket team', 0.8031073808670044),
 ('sleeping pikachu', 0.797552227973938),
 ('cute pikachu', 0.792954683303833),
 ('sponzar', 0.7915909290313721),
 ('japanise', 0.7874931693077087),
 ('broak', 0.7773244380950928),
 ('misty brock', 0.7767468094825745),
 ('phanpy', 0.7735733389854431),
 ('kaosvortex', 0.7722904682159424)]

In [12]:
word2vec_cbow.most_similar(positive=['pokemon'])

[('pikachu', 0.7273429036140442),
 ('bulbasaur', 0.6967936158180237),
 ('pokeball', 0.6956156492233276),
 ('poke', 0.6887432336807251),
 ('ash', 0.6582557559013367),
 ('got to catch em all', 0.6527172327041626),
 ('scyther', 0.6452850103378296),
 ('got to catch them all', 0.6432250142097473),
 ('pok mon', 0.6415690779685974),
 ('jigglypuff', 0.6400002241134644)]

In [13]:
word2vec_sg.most_similar(positive=['pokemon'])

[('pikachu', 0.8366302251815796),
 ('gallade', 0.7769574522972107),
 ('kogepan', 0.7640161514282227),
 ('the ten commandments of marching band', 0.7596938610076904),
 ('pikache', 0.757293701171875),
 ('tare panda', 0.7562474012374878),
 ('number 25', 0.7558513283729553),
 ('koffin', 0.7539600729942322),
 ('pokemon crossover', 0.7499330639839172),
 ('arcenine', 0.7487433552742004)]

In [14]:
word2vec_cbow.most_similar(positive=['bdsm'])

[('kinky', 0.8701714277267456),
 ('bondage', 0.8693689107894897),
 ('fetish', 0.8475905656814575),
 ('s m', 0.8423230648040771),
 ('submissive', 0.8085852861404419),
 ('kink', 0.8054283857345581),
 ('female domination', 0.8045685291290283),
 ('s and m', 0.7981154918670654),
 ('nymphet', 0.7925896644592285),
 ('leather', 0.7778752446174622)]

In [15]:
word2vec_sg.most_similar(positive=['bdsm'])

[('bondage', 0.8044426441192627),
 ('kama sutra', 0.7740716934204102),
 ('good girl', 0.7700810432434082),
 ('mr bondage', 0.7523753643035889),
 ('need a spanking', 0.7491353154182434),
 ('d s', 0.7412813305854797),
 ('honkyourhorny com', 0.7385274171829224),
 ('kamasutra', 0.7375245690345764),
 ('b d s m', 0.7314965724945068),
 ('sith grotto', 0.7291421890258789)]

In [21]:
word2vec_cbow.most_similar(positive=['pussy'])

[('vagina', 0.66108238697052),
 ('cock', 0.6522481441497803),
 ('durex', 0.6278104782104492),
 ('ass', 0.6123033761978149),
 ('grubbanax', 0.6062963008880615),
 ('dick', 0.5889889597892761),
 ('naughty', 0.5839887857437134),
 ('pepsi cola', 0.5758708119392395),
 ('penis', 0.5731520652770996),
 ('cunt', 0.5700767040252686)]

In [20]:
word2vec_sg.most_similar(positive=['pussy'])

[('totas', 0.724101185798645),
 ('tota', 0.7143247127532959),
 ('rabbit vibe', 0.6999883055686951),
 ('minge', 0.6945575475692749),
 ('durex', 0.6936394572257996),
 ('vaginas', 0.6920632123947144),
 ('my pussy my rules', 0.6883783340454102),
 ('pussyman', 0.6808770298957825),
 ('fanny', 0.6782764196395874),
 ('sex toys', 0.6771144270896912)]

In [25]:
word2vec_sg.most_similar(positive=['nordic', 'viking', 'norse'])

[('dragon boat', 0.813959538936615),
 ('viking berserker', 0.81053227186203),
 ('the vikings', 0.8100918531417847),
 ('viking hammer', 0.8074697256088257),
 ('norse god', 0.8045765161514282),
 ('t shirt winter is coming', 0.8009395003318787),
 ('astrau', 0.7943754196166992),
 ('celtic hammer club', 0.7937148809432983),
 ('ragnar k', 0.7936811447143555),
 ('norseman', 0.7917630672454834)]

In [28]:
v1 = word2vec_sg['nordic']
v2 = word2vec_sg['viking']
v3 = word2vec_sg['norse']

In [29]:
import numpy as np

In [33]:
np.mean([v1,v2,v3], keepdims=True)

array([[ 0.00261299]], dtype=float32)

In [34]:
word2vec.dot(v1,v2)

29.368214

In [38]:
m = v1 * v2 * v3

In [39]:
word2vec_sg.similar_by_vector(m)

[('norse', 0.7565549612045288),
 ('viking', 0.7482388019561768),
 ('nordic', 0.73396897315979),
 ('norseman', 0.6900303363800049),
 ('norse blood', 0.6695829033851624),
 ('norsemen', 0.6693034172058105),
 ('viking berserker', 0.6637814044952393),
 ('wiking', 0.6571587324142456),
 ('norse god', 0.6539167165756226),
 ('thors', 0.6537144780158997)]

In [60]:
search_queries = []

In [61]:
search_tokens_tags = []

In [62]:
with open('user_search_keywords.tsv','r') as inpFile:
    lines = inpFile.readlines()
    for line in lines[1:1000]:
        search_tokens = line.split("\t")
        search_tokens = search_tokens[1].replace("\n", "")
        search_tokens =  search_tokens.split(",")
        search_queries.append(search_tokens)
        token_list_that_has_similarity = []
        for token in search_tokens:
            try:
                #print("Finding relevant products for token %s"%(token,))
                a = word2vec.most_similar(positive=[token])
                token_list_that_has_similarity.append(token)
            except:
                print("No relevant documents found for %s"%(token,))
        search_tokens_tags.append(token_list_that_has_similarity)
        

No relevant documents found for norse viking
No relevant documents found for finess
No relevant documents found for lucian lol
No relevant documents found for pryda
No relevant documents found for iceage
No relevant documents found for elias bender
No relevant documents found for the knife band
No relevant documents found for grimes band
No relevant documents found for claire elise boucher
No relevant documents found for grimes visions
No relevant documents found for iceage
No relevant documents found for grimes visions
No relevant documents found for iceage band
No relevant documents found for t-shirts
No relevant documents found for laptop-skins
No relevant documents found for roswell michael and maria
No relevant documents found for roswell high
No relevant documents found for roswell michael
No relevant documents found for roswell tv show
No relevant documents found for t-shirts
No relevant documents found for phantom of opera
No relevant documents found for audley hep penuryt-shir

In [38]:
len(search_tokens_tags)

999

In [ ]:
for token in search_tokens_tags[:10]:
    print(word2vec_cbow.most_similar(positive=token))
    print(word2vec_sg.most_similar(positive=token))

In [ ]:
word2vec_sg.

In [79]:
for token in search_queries[:10]:
    print(token)
    print(word2vec_cbow.index2word(token))

['norse viking', 'nordic']


TypeError: 'list' object is not callable

In [46]:
from gensim.models import Doc2Vec
from gensim.models import doc2vec

In [45]:
labelled_sentences = []

In [47]:
counter = 0

In [50]:
for tags in cleaned_tags:
    labelled_sentences.append(doc2vec.LabeledSentence(tags, 'SENT_'+str(counter)))
    counter = counter + 1

In [53]:
labelled_sentences

[LabeledSentence(words=['rocket', 'uniform', 'villain', 'grunt', 'team rocket', 'team rocket grunt', 'pokemon'], tags='SENT_0'),
 LabeledSentence(words=['hfil', 'home for infinite losers', 'dbz', 'dragonball z', '4kids', 'anime', 'japan', 'manga', 'dub', 'dragon ball', 'goku', 'fight', 'fighting', 'saiyan', 'super saiyan', 'akira toriyama', 'dragon', 'ball'], tags='SENT_1'),
 LabeledSentence(words=['pikachu', 'pokemon', 'cute', 'davinci', 'vitruvian man', 'cartoon', 'cartoons', 'geek', 'nerd', 'movies', 'tv'], tags='SENT_2'),
 LabeledSentence(words=['pokemon', 'evolution', 'red', 'fire', 'starter'], tags='SENT_3'),
 LabeledSentence(words=['how to train your dragon', 'dragon', 'toothless', 'pokemon', 'pokeball', 'nightfury', 'funny', 'cool', 'vector', 'geek', 'retro', 'cute'], tags='SENT_4'),
 LabeledSentence(words=['magikarp', 'pokemon', 'funny', 'cartoon', 'graphic', 'cute', 'vintage', 'geek', 'vector', 'girl', 'artistic', 'arty', 'contemporary', 'orange', 'fish', 'evolve'], tags='SEN

In [64]:
d2v = Doc2Vec(documents=labelled_sentences, size=100, window=8, workers=4, min_count=1)

In [65]:
d2v.vocab

{'': <gensim.models.word2vec.Vocab at 0x108373a58>,
 'dinah laurel lance': <gensim.models.word2vec.Vocab at 0x142c4a400>,
 'sixx': <gensim.models.word2vec.Vocab at 0x108373940>,
 'tibetan singing bowl': <gensim.models.word2vec.Vocab at 0x108373ac8>,
 'owelia': <gensim.models.word2vec.Vocab at 0x108373cc0>,
 'cybor': <gensim.models.word2vec.Vocab at 0x108373a90>,
 'brand the goonies': <gensim.models.word2vec.Vocab at 0x108373b38>,
 'daily motivation': <gensim.models.word2vec.Vocab at 0x108373b70>,
 'black frost': <gensim.models.word2vec.Vocab at 0x108373ba8>,
 'katara': <gensim.models.word2vec.Vocab at 0x108373be0>,
 'fang': <gensim.models.word2vec.Vocab at 0x108373c18>,
 'electronic music': <gensim.models.word2vec.Vocab at 0x108373cf8>,
 'smeg': <gensim.models.word2vec.Vocab at 0x1184adcc0>,
 'furry fandom': <gensim.models.word2vec.Vocab at 0x108373d30>,
 'the elder scrolls v': <gensim.models.word2vec.Vocab at 0x108373d68>,
 'nien nunb': <gensim.models.word2vec.Vocab at 0x1180e6c50>,
 

In [68]:
from gensim.models import doc2vec

In [78]:
for token in search_queries[:10]:
    print(token)
    m = d2v.infer_vector(token)
    print(d2v.most_similar(positive=[m]))

['norse viking', 'nordic']
[('new jersey', 0.7316958904266357), ('arse', 0.6830275058746338), ('square', 0.6813724040985107), ('basquiat crown', 0.6359907984733582), ('gangponieszomg', 0.6339077353477478), ('andy evans', 0.6187152862548828), ('heights', 0.592673122882843), ('exu', 0.5823046565055847), ('jean michel baquiat shirt', 0.5821531414985657), ('kpop music', 0.5819041728973389)]
['finess', 'trap']
[('clap', 0.7136750221252441), ('bruno', 0.7024455666542053), ('nugget', 0.6861792206764221), ('holy other', 0.6707488298416138), ('nic cage', 0.6590348482131958), ('fantasy football', 0.6328897476196289), ('gone in 60 seconds', 0.6247032880783081), ('muethbooth', 0.6163164377212524), ('ooooo', 0.6151666641235352), ('cucco', 0.6000058054924011)]
['league of legends', 'lucian', 'lucian lol']
[('knight solaire of astora', 0.5866432189941406), ('pensioners', 0.581718921661377), ('attack damage', 0.5597900152206421), ('theonlyradius', 0.5573095083236694), ('dragon fist lee sin', 0.5459936